[데이터셋](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/overview)


[사용](https://www.kaggle.com/code/tushargoel04/store-sales-time-series-analysis/comments)

# 목표

- 시계열 예측을 사용하여 대규모 에콰도르 기반 식료품 소매업체인 Corporación Favorita의 매장 판매 예측
- 구체적으로 여러 Favorita 매장에서 판매되는 수천 개의 상품에 대한 단위 판매량을 더 정확하게 예측하는 모델 구축

# 영향

- 식료품 점주는 얼마나 많은 재고를 구입할지 신중하게 결정해야 한다. 조금 과다 예측하면 식료품이 과잉 재고되어 상품이 손상된다. 조금 부족하게 추정하면 인기 있는 상품이 빨리 품절되어 수익 손실과 고객의 불만이 발생할 수 있다.
- 머신 러닝을 통한 더 정확한 예측은 소매업체가 고객을 만족시키기 위해 정확한 시간에 정확한 제품을 딱 맞추는 데 도움이 될 수 있다.
- 식료품 점포에서 더 정확한 예측은 과잉 재고로 인한 음식 폐기물을 줄이고 고객 만족도를 향상시킬 수 있다.

# 흐름
- 1~12: 각각 데이터 시각화 > 인사이트 얻음(ex: 언제 많이 팔리는지, 연간 거래량 추이 등등)
- 13: 모델링
  - train, val 분리 (train, test data는 주어짐)
  - transformer
    - XGBoost
    - 파이프라인 구축
  - k-fold로 교차검증

# 1.Data Load

In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/sesac/Project

/content/drive/MyDrive/sesac/Project


In [ ]:
!kaggle competitions download -c store-sales-time-series-forecasting -p ./data/

In [ ]:
!unzip ./data/store-sales-time-series-forecasting.zip -d ./data/store-sales-time-series-forecasting

# 2.Preprocessing

In [ ]:
!pip install feature_engine

In [ ]:
# BASE
import numpy as np
import pandas as pd
import os
import calendar

# DATA VISUALIZATION
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.ticker as ticker

# 3.Import data

In [ ]:
# dataframe으로
holidays = pd.read_csv('./data/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('./data/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('./data/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('./data/store-sales-time-series-forecasting/transactions.csv')

train = pd.read_csv('./data/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('./data/store-sales-time-series-forecasting/test.csv')

- 한 번에 다 merge 했더니 결측치 너무 많아서 안 합치는 방식으로 진행

In [ ]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [ ]:
# object -> Datetime 변경
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)
oil["date"] = pd.to_datetime(oil.date)
holidays["date"] = pd.to_datetime(holidays.date)

In [ ]:
# 바꾼 거 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [ ]:
# 다른 거 Data type 변경
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

In [ ]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int8  
dtypes: int64(1), int8(1), object(3)
memory usage: 1.9+ KB


In [ ]:
# shape 확인
print(f'Number of data_oil samples: {oil.shape}')
print(f'Number of train_data samples: {train.shape}')
print(f'Number of test_data samples: {test.shape}')
print(f'Number of data_holi samples: {holidays.shape}')
print(f'Number of data_store samples: {stores.shape}')
print(f'Number of data_trans samples: {transactions.shape}')

Number of data_oil samples: (1218, 2)
Number of train_data samples: (3000888, 6)
Number of test_data samples: (28512, 5)
Number of data_holi samples: (350, 6)
Number of data_store samples: (54, 5)
Number of data_trans samples: (83488, 3)


# 4.Train

In [ ]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float32       
 5   onpromotion  float16       
dtypes: datetime64[ns](1), float16(1), float32(1), int64(2), object(1)
memory usage: 108.8+ MB


In [ ]:
train.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3000888.0
mean,1.500444e+06,2.750000e+01,3.577758e+02,NaN
std,8.662819e+05,1.558579e+01,1.101998e+03,NaN
min,0.000000e+00,1.000000e+00,0.000000e+00,0.0
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.0
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.0
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.0
max,3.000887e+06,5.400000e+01,1.247170e+05,741.0


In [ ]:
train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [ ]:
print(train.family.unique())
print(len(train.family.unique()))

train['family'] = train['family'].astype('category')
train['family_category'] = train['family'].cat.codes
# 'family'열의 값을 카테고리형으로 인코딩하여 'family_category'열에 저장

['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD']
33


In [ ]:
family_category = dict(zip(train['family_category'], train['family']))

# 'train'이라는 데이터프레임에서 'family_category' 열의 값을 키(key)로,
# 'family' 열의 값을 값(value)으로 갖는 딕셔너리를 생성하는 코드
# 딕셔너리의 키는 'family_category' 열의 각 행의 값이 되고, 값은 해당 행의 'family' 열의 값이 됨
# 이렇게 생성된 딕셔너리는 데이터프레임의 'family_category' 열의 값과 'family' 열의 값을 매핑하여 저장

family_category

{0: 'AUTOMOTIVE',
 1: 'BABY CARE',
 2: 'BEAUTY',
 3: 'BEVERAGES',
 4: 'BOOKS',
 5: 'BREAD/BAKERY',
 6: 'CELEBRATION',
 7: 'CLEANING',
 8: 'DAIRY',
 9: 'DELI',
 10: 'EGGS',
 11: 'FROZEN FOODS',
 12: 'GROCERY I',
 13: 'GROCERY II',
 14: 'HARDWARE',
 15: 'HOME AND KITCHEN I',
 16: 'HOME AND KITCHEN II',
 17: 'HOME APPLIANCES',
 18: 'HOME CARE',
 19: 'LADIESWEAR',
 20: 'LAWN AND GARDEN',
 21: 'LINGERIE',
 22: 'LIQUOR,WINE,BEER',
 23: 'MAGAZINES',
 24: 'MEATS',
 25: 'PERSONAL CARE',
 26: 'PET SUPPLIES',
 27: 'PLAYERS AND ELECTRONICS',
 28: 'POULTRY',
 29: 'PREPARED FOODS',
 30: 'PRODUCE',
 31: 'SCHOOL AND OFFICE SUPPLIES',
 32: 'SEAFOOD'}

In [ ]:
train['family'].value_counts()

AUTOMOTIVE                    90936
HOME APPLIANCES               90936
SCHOOL AND OFFICE SUPPLIES    90936
PRODUCE                       90936
PREPARED FOODS                90936
POULTRY                       90936
PLAYERS AND ELECTRONICS       90936
PET SUPPLIES                  90936
PERSONAL CARE                 90936
MEATS                         90936
MAGAZINES                     90936
LIQUOR,WINE,BEER              90936
LINGERIE                      90936
LAWN AND GARDEN               90936
LADIESWEAR                    90936
HOME CARE                     90936
HOME AND KITCHEN II           90936
BABY CARE                     90936
HOME AND KITCHEN I            90936
HARDWARE                      90936
GROCERY II                    90936
GROCERY I                     90936
FROZEN FOODS                  90936
EGGS                          90936
DELI                          90936
DAIRY                         90936
CLEANING                      90936
CELEBRATION                 

In [ ]:
train['day_of_week'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year

In [ ]:
train.tail()

,id,date,store_nbr,family,sales,onpromotion,family_category,day_of_week,month,year
3000883,3000883,2017-08-15,9,POULTRY,438.132996,0.0,28,1,8,2017
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.552994,1.0,29,1,8,2017
3000885,3000885,2017-08-15,9,PRODUCE,2419.729004,148.0,30,1,8,2017
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000000,8.0,31,1,8,2017
3000887,3000887,2017-08-15,9,SEAFOOD,16.000000,0.0,32,1,8,2017


In [ ]:
## 그래프 안 보일 때 실행하고 런타임 다시 시작
# import plotly.io as pio
# pio.renderers.default = "colab"

In [ ]:
data_grouped_day = train.groupby(['day_of_week']).mean(numeric_only=True)['sales']
# 'train' 데이터프레임을 'day_of_week' 열을 기준으로 그룹화한 후,
# 각 그룹의 평균을 계산하여 'sales' 열만을 선택하여 반환
# -> 수치형 데이터만을 고려하여 평균을 계산하고, 결과는 'data_grouped_day' 변수에 저장

data_grouped_month = train.groupby(['month']).mean(numeric_only=True)['sales']
data_grouped_year = train.groupby(['year']).mean(numeric_only=True)['sales']

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Sales - Day of Week', 'Sales - Month', 'Sales - Year'))

# Day of Week
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fig.add_trace(go.Bar(x=day_names, y=data_grouped_day, marker=dict(color=data_grouped_day, colorscale='Viridis')), row=1, col=1)
fig.update_xaxes(title_text='Day of Week', row=1, col=1, tickangle=-90)
fig.update_yaxes(title_text='Mean Sales', row=1, col=1, title_standoff=1)

# Month
month_names = [calendar.month_name[i] for i in range(1, 13)]
fig.add_trace(go.Bar(x=month_names, y=data_grouped_month, marker=dict(color=data_grouped_month, colorscale='Viridis')), row=1, col=2)
fig.update_xaxes(title_text='Month', row=1, col=2, tickangle=-90)
fig.update_yaxes(title_text='Mean Sales', row=1, col=2, title_standoff=1)

# Year
fig.add_trace(go.Bar(x=data_grouped_year.index, y=data_grouped_year, marker=dict(color=data_grouped_year, colorscale='Viridis')), row=1, col=3)
fig.update_xaxes(title_text='Year', row=1, col=3, tickangle=-90, title_standoff=48, tickvals=data_grouped_year.index, ticktext=data_grouped_year.index)
fig.update_yaxes(title_text='Mean Sales', row=1, col=3, title_standoff=1)

fig.update_layout(height=400, width=1000, showlegend=False)

fig.show()

- 가장 많이 판 요일: 토요일, 일요일
- 가장 많이 판 월: 12월
- 연간 거래 건수는 매년 증가함

# 5.Transactions

In [ ]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
transactions['store_nbr'].unique()

array([25,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 39,
       40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 54, 36, 53, 20, 29, 21,
       42, 22, 52])

In [ ]:
store_counts = transactions['store_nbr'].value_counts().sort_index()
# 'transactions' 데이터프레임의 'store_nbr' 열에서 각 상점 번호의 등장 횟수를 세어서,
# 상점 번호를 기준으로 오름차순으로 정렬한 후, 그 결과를 'store_counts' 변수에 저장
# -> 'store_counts' 변수에는 각 상점 번호의 등장 횟수가 포함된 Series가 저장

fig = go.Figure()

bars = fig.add_trace(go.Bar(x=store_counts.index, y=store_counts.values, marker_color=store_counts.values, marker_colorscale='RdYlBu'))
fig.update_layout(title_text="Total Transactions per Store Number", xaxis_title="Store Number", yaxis_title="Number of Transactions") # 스토어 번호당 총 거래 수
fig.update_xaxes(tickangle=-90, title_standoff=20, tickvals=store_counts.index, ticktext=store_counts.index)
fig.update_yaxes(title_standoff=5)

fig.update_layout(height=400, width=1000, showlegend=False)

fig.show()

- 매장별 거래액을 나타내는 그래프
- 20, 21, 22, 29, 42, 52, 53번 가게의 거래액이 적음

In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [ ]:
transactions.describe()

,store_nbr,transactions
count,83488.000000,83488.000000
mean,26.939237,1694.602158
std,15.608204,963.286644
min,1.000000,5.000000
25%,13.000000,1046.000000
50%,27.000000,1393.000000
75%,40.000000,2079.000000
max,54.000000,8359.000000


In [ ]:
transactions.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [ ]:
transactions['day_of_week'] = transactions['date'].dt.dayofweek
transactions['month'] = transactions['date'].dt.month
transactions['year'] = transactions['date'].dt.year

In [ ]:
transactions.head()

,date,store_nbr,transactions,day_of_week,month,year
0,2013-01-01,25,770,1,1,2013
1,2013-01-02,1,2111,2,1,2013
2,2013-01-02,2,2358,2,1,2013
3,2013-01-02,3,3487,2,1,2013
4,2013-01-02,4,1922,2,1,2013


In [ ]:
data_grouped_day = transactions.groupby(['day_of_week']).mean()['transactions']
data_grouped_month = transactions.groupby(['month']).mean()['transactions']
data_grouped_year = transactions.groupby(['year']).mean()['transactions']

fig = make_subplots(rows=1, cols=3, subplot_titles=('Transactions - Day of Week', 'Transactions - Month', 'Transactions - Year'))

# Day of Week
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
fig.add_trace(go.Bar(x=day_names, y=data_grouped_day, marker=dict(color=data_grouped_day, colorscale='Viridis')), row=1, col=1)
fig.update_xaxes(title_text='Day of Week', row=1, col=1, tickangle=-90)
fig.update_yaxes(title_text='Mean Transactions', row=1, col=1, title_standoff=1)

# Month
month_names = [calendar.month_name[i] for i in range(1, 13)]
fig.add_trace(go.Bar(x=month_names, y=data_grouped_month, marker=dict(color=data_grouped_month, colorscale='Viridis')), row=1, col=2)
fig.update_xaxes(title_text='Month', row=1, col=2, tickangle=-90)
fig.update_yaxes(title_text='Mean Transactions', row=1, col=2, title_standoff=1)

# Year
fig.add_trace(go.Bar(x=data_grouped_year.index, y=data_grouped_year, marker=dict(color=data_grouped_year, colorscale='Viridis')), row=1, col=3)
fig.update_xaxes(title_text='Year', row=1, col=3, tickangle=-90, title_standoff=48, tickvals=data_grouped_year.index, ticktext=data_grouped_year.index)
fig.update_yaxes(title_text='Mean Transactions', row=1, col=3, title_standoff=1)

fig.update_layout(height=400, width=1000, showlegend=False)

fig.show()

<ipython-input-270-0a3e67c29c9b>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-270-0a3e67c29c9b>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-270-0a3e67c29c9b>:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



- 가장 많이 판 요일: 토요일, 일요일
- 가장 많이 판 월: 12월
- 연간 거래 건수: 거의 동일한 수준 유지

In [ ]:
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.1, subplot_titles=("Transactions by Month", "Transactions by Year"))

# 1. Transactions by Month
colors_month = [
    'red', 'green', 'blue',
    'magenta', 'yellow', 'brown',
    'pink', 'darkgrey', 'orange',
    'darkblue', 'olive', 'purple'
]

years = transactions['date'].dt.year.unique()
for year in years:
    trace_data = []
    for month in range(1, 13):
        data_month = transactions[(transactions['date'].dt.year == year) & (transactions['date'].dt.month == month)]
        trace = go.Box(x=[f"{calendar.month_abbr[month]}-{year}"] * len(data_month['transactions']), y=data_month['transactions'],
                       boxpoints='outliers', jitter=0.4, pointpos=0,
                       marker=dict(color=colors_month[month-1]),
                       name=f"{calendar.month_abbr[month]}-{year}")
        trace_data.append(trace)

    fig.add_traces(trace_data, rows=[1] * len(trace_data), cols=[1] * len(trace_data))

fig.update_xaxes(title_text="Month-Year", tickvals=[f"{calendar.month_abbr[month]}-{year}" for year in years for month in range(1, 13)],
                 ticktext=[f"{calendar.month_abbr[month]}-{year}" for year in years for month in range(1, 13)], row=1, col=1)
fig.update_yaxes(title_text="Transactions", row=1, col=1)


# 2. Transactions by Year
colors_year = ['skyblue', 'hotpink', 'violet',
               'lawngreen', 'firebrick']

for i, year in enumerate(transactions['date'].dt.year.unique()):
    data_year = transactions[transactions['date'].dt.year == year]
    fig.add_trace(go.Box(x=data_year['date'].dt.year, y=data_year['transactions'],
                         boxpoints='all', jitter=0.6, pointpos=-1.8,
                         marker=dict(color=colors_year[i])),
                  row=2, col=1)

fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_yaxes(title_text="Transactions", row=2, col=1)

fig.update_layout(height=1300,width=1000, showlegend=False)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

- 월별 거래량의 상자 그림
- 매년 12월이 가장 거래량이 많음
- 2013~2016년까지는 비슷하였으나 2017년에 감소함

# 6.Oil Prices

In [ ]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [ ]:
oil.tail()

,date,dcoilwtico
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96
1217,2017-08-31,47.26


In [ ]:
oil.info()
# dcoilwtico: 원유 가격 관련 지표

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1175 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB


In [ ]:
oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [ ]:
oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [ ]:
oil['dcoilwtico'].unique()

array([   nan,  93.14,  92.97,  93.12,  93.2 ,  93.21,  93.08,  93.81,
        93.6 ,  94.27,  93.26,  94.28,  95.49,  95.61,  96.09,  95.06,
        95.35,  95.15,  95.95,  97.62,  97.98,  97.65,  97.46,  96.21,
        96.68,  96.44,  95.84,  95.71,  97.01,  97.48,  97.03,  97.3 ,
        96.69,  94.92,  92.79,  92.74,  92.63,  92.84,  92.03,  90.71,
        90.13,  90.88,  90.47,  91.53,  92.01,  92.07,  92.44,  92.47,
        93.03,  93.49,  93.71,  92.46,  93.41,  94.55,  95.99,  96.53,
        97.24,  97.1 ,  97.23,  95.02,  92.76,  93.36,  94.18,  94.59,
        93.44,  91.23,  88.75,  88.73,  86.65,  87.83,  88.04,  88.81,
        89.21,  91.07,  93.27,  94.09,  93.22,  90.74,  93.7 ,  95.25,
        95.8 ,  95.28,  96.24,  95.81,  94.76,  93.96,  93.95,  94.85,
        95.72,  96.29,  95.55,  93.98,  94.12,  93.84,  94.65,  93.13,
        93.57,  91.93,  93.66,  94.71,  96.11,  95.82,  95.5 ,  95.98,
        96.66,  97.83,  97.86,  98.46,  98.24,  94.89,  95.07,  95.47,
      

In [ ]:
oil.fillna(method='bfill', inplace = True)

In [ ]:
# 최종으로 null값 존재 확인
oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

In [ ]:
oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=oil["date"], y=oil["dcoilwtico"], mode="lines", name="dcoilwtico", line_color="blue"))
fig.update_layout(title="dcoilwtico", xaxis_title="date", yaxis_title="dcoilwtico",height=670)
fig.show()

- 유가는 2013년 하반기에 가장 높았으나 2014년부터 하락하기 시작하여 2016년에 최저치를 기록함. 이후 계속 상승하여 40~60달러의 가격대 유지

In [ ]:
fig = make_subplots(rows=2, cols=1, subplot_titles=("Crude Oil Prices by Month for Each Year", "Crude Oil Prices by Year"))

# 1. Crude Oil Prices by Month for Each Year
years = oil['date'].dt.year.unique()
colors_month = [
    'red', 'green', 'blue',
    'magenta', 'yellow', 'brown',
    'pink', 'darkgrey', 'orange',
    'darkblue', 'olive', 'purple'
]

for i, year in enumerate(years):
    data_year = oil[oil['date'].dt.year == year]
    for month in range(1, 13):
        data_month = data_year[data_year['date'].dt.month == month]
        fig.add_trace(go.Box(x=[f"{calendar.month_abbr[month]} - {year}"] * len(data_month),
                             y=data_month['dcoilwtico'],
                             boxpoints='outliers', jitter=0.3, pointpos=0,
                             marker=dict(color=colors_month[month-1], size=2),
                             name=f"{calendar.month_abbr[month]}"),
                      row=1, col=1)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Crude Oil Price", row=1, col=1)

# 2. Crude Oil Prices by Year
colors_year = ['skyblue', 'hotpink', 'violet',
               'lawngreen', 'firebrick']

for i, year in enumerate(years):
    data_year = oil[oil['date'].dt.year == year]
    fig.add_trace(go.Box(x=data_year['date'].dt.year, y=data_year['dcoilwtico'], boxpoints='all', jitter=0.3, pointpos=-1.8,
                    marker=dict(color=colors_year[i]), name="Crude Oil Prices by Year"), row=2, col=1)
fig.update_xaxes(title_text="Year", row=2, col=1)
fig.update_yaxes(title_text="Crude Oil Price", row=2, col=1)

fig.add_trace(go.Scatter(x=oil["date"], y=oil["dcoilwtico"], mode="lines", name="dcoilwtico", line_color="blue"), row=2, col=1)

fig.update_layout(showlegend=False, height=1300, width=1000)

fig.show()

- 연도별 원유 가격의 상자 그림
- 추세선과 거의 비슷한 결과

# 7.Stores

In [ ]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
stores['type'].unique()

array(['D', 'B', 'C', 'E', 'A'], dtype=object)

In [ ]:
stores['state'].unique()

array(['Pichincha', 'Santo Domingo de los Tsachilas', 'Cotopaxi',
       'Chimborazo', 'Imbabura', 'Bolivar', 'Pastaza', 'Tungurahua',
       'Guayas', 'Santa Elena', 'Los Rios', 'Azuay', 'Loja', 'El Oro',
       'Esmeraldas', 'Manabi'], dtype=object)

In [ ]:
stores['cluster'].unique()

array([13,  8,  9,  4,  6, 15,  7,  3, 12, 16,  1, 10,  2,  5, 11, 14, 17],
      dtype=int8)

In [ ]:
stores['cluster'].value_counts()

3     7
6     6
10    6
15    5
13    4
14    4
11    3
4     3
8     3
1     3
9     2
7     2
2     2
12    1
5     1
16    1
17    1
Name: cluster, dtype: int64

In [ ]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int8  
dtypes: int64(1), int8(1), object(3)
memory usage: 1.9+ KB


In [ ]:
stores.describe()

,store_nbr,cluster
count,54.000000,54.000000
mean,27.500000,8.481481
std,15.732133,4.693395
min,1.000000,1.000000
25%,14.250000,4.000000
50%,27.500000,8.500000
75%,40.750000,13.000000
max,54.000000,17.000000


In [ ]:
stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

# Cluster
fig_cluster = px.bar(stores['cluster'].value_counts(), labels={'value': 'Count', 'index': 'Cluster'})
fig_cluster.update_layout(title='Cluster')

# Type
fig_type = px.bar(stores['type'].value_counts(), labels={'value': 'Count', 'index': 'Type'})
fig_type.update_layout(title='Type')

# State
fig_state = px.bar(stores['state'].value_counts(), labels={'value': 'Count', 'index': 'State'})
fig_state.update_layout(title='State')

# City
fig_city = px.bar(stores['city'].value_counts(), labels={'value': 'Count', 'index': 'City'})
fig_city.update_layout(title='City')

# Creating subplots
fig_combined = make_subplots(rows=2, cols=2, subplot_titles=['Cluster', 'Type', 'State', 'City'])

# Adding traces to subplots using marker parameter
fig_combined.add_trace(fig_cluster.data[0].update(marker=dict(color=stores['cluster'].value_counts(),colorscale='Viridis')), row=1, col=1)
fig_combined.add_trace(fig_type.data[0].update(marker=dict(color=stores['type'].value_counts(),colorscale='Viridis')), row=1, col=2)
fig_combined.add_trace(fig_state.data[0].update(marker=dict(color=stores['state'].value_counts(),colorscale='Viridis')), row=2, col=1)
fig_combined.add_trace(fig_city.data[0].update(marker=dict(color=stores['city'].value_counts(),colorscale='Viridis')), row=2, col=2)

# Rotate labels for 'City' and 'State'
fig_combined.update_xaxes(tickmode='linear', row=1, col=1)
fig_combined.update_xaxes(tickangle=-90, row=2, col=1)  # Rotate 'State'
fig_combined.update_xaxes(tickangle=-90, row=2, col=2)  # Rotate 'City'

# Decrease the interval on y-axis
tick_interval = 2
fig_combined.update_yaxes(tickmode='linear', row=1, col=1)
fig_combined.update_yaxes(dtick=tick_interval , row=1, col=2)
fig_combined.update_yaxes(dtick=tick_interval , row=2, col=1)
fig_combined.update_yaxes(dtick=tick_interval , row=2, col=2)

fig_combined.update_layout(height=800, width=900, showlegend=False, margin=dict(l=0, r=0, t=20, b=0))

fig_combined.show()

- cluster, type, state, city column의 값에 따른 막대 그래프
- 빈도수 확인 가능

# 8.Holidays

In [ ]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [ ]:
holidays['locale_name'].unique()

array(['Manta', 'Cotopaxi', 'Cuenca', 'Libertad', 'Riobamba', 'Puyo',
       'Guaranda', 'Imbabura', 'Latacunga', 'Machala', 'Santo Domingo',
       'El Carmen', 'Cayambe', 'Esmeraldas', 'Ecuador', 'Ambato',
       'Ibarra', 'Quevedo', 'Santo Domingo de los Tsachilas',
       'Santa Elena', 'Quito', 'Loja', 'Salinas', 'Guayaquil'],
      dtype=object)

In [ ]:
holidays['locale'].unique()

array(['Local', 'Regional', 'National'], dtype=object)

In [ ]:
holidays['type'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [ ]:
holidays['type'].value_counts()

Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: type, dtype: int64

In [ ]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


In [ ]:
holidays.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [ ]:
day_type = holidays[['date', 'type']]
avg_sales = train.groupby('date').agg({'sales': 'mean'}).reset_index()

# Merge the dataframes and drop NaN values
df = pd.merge_asof(day_type, avg_sales, on='date')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Calculate average sales by holiday type
df_1 = df.groupby(['type']).mean()['sales']
average_holiday_sales = df_1.mean()

print(f'Average holiday sales is {average_holiday_sales}')

fig = px.bar(df_1, x=df_1.index, y=df_1, color=df_1.index, labels={'y': 'Mean Sales'})
fig.update_layout(title='Average Holiday Sales', xaxis_title='Holiday Type', yaxis_title='Mean Sales', showlegend=False)

fig.show()

Average holiday sales is 425.4221496582031


<ipython-input-230-a5e4dd335a1b>:10: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



- 휴일 유형별 평균 판매량을 나타내는 막대 그래프

In [ ]:
# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=['Holiday Types', 'Holiday Locales'])

# Plotting for 'Holiday Types'
fig_types = px.bar(holidays['type'].value_counts(), labels={'value': 'Count', 'index': 'Holiday Types'}, color_discrete_map={'type': px.colors.qualitative.Set1})
fig_types.update_layout(title_text='Holiday Types')
fig_types.update_xaxes(tickangle=90, row=1, col=1)
fig.add_trace(fig_types['data'][0], row=1, col=1)


# Plotting for 'Holiday Locales'
fig_locales = px.bar(holidays['locale'].value_counts(), labels={'value': 'Count', 'index': 'Holiday Locales'}, color_discrete_map={'locale': px.colors.qualitative.Set2})
fig_locales.update_layout(title_text='Holiday Locales')
fig.add_trace(fig_locales['data'][0], row=1, col=2)

# Update layout
fig.update_layout(width=950, showlegend=False)

# Show the plot
fig.show()

- type 열에서 각 휴일 유형의 빈도를 나타내는 그래프
- locale 열에서 각 휴일 지역의 빈도를 나타내는 그래프

# 9.Visualizations

- most sold items

In [ ]:
data_grouped_family_types = train.groupby(['family_category']).mean()[['sales', 'onpromotion']]

# Calculate percentage
data_grouped_family_types['%_s'] = 100 * data_grouped_family_types['sales'] / data_grouped_family_types['sales'].sum()
data_grouped_family_types['%_s'] = data_grouped_family_types['%_s'].round(decimals=3)

# Create labels
labels = [f"{category} - {percent:.2f}%" for category, percent in zip(family_category.values(), data_grouped_family_types['%_s'])]

# Create a Pie chart
fig = go.Figure(go.Pie(
    labels=labels,
    values=data_grouped_family_types['%_s'],
    textinfo='none',
    marker=dict(colors=px.colors.sequential.Rainbow),
    sort=True,  # Keep the order as in the DataFrame
))

fig.update_layout(
    title="Sales Distribution by Family Category",
    legend=dict(orientation="v", x=-0.8, y=0.2), height=950, width=900
)

fig.show()

<ipython-input-232-b3f0b4134e7a>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



- 판매량을 카테고리별로 표시하여 각 카테고리가 전체 판매량에서 차지하는 백분율을 시각적으로 보여줌

---

- On promotion of items

In [ ]:
data_grouped_family_types = train.groupby(['family_category']).mean()[['sales', 'onpromotion']]

# Calculate percentage
data_grouped_family_types['%_p'] = 100 * data_grouped_family_types['onpromotion'] / data_grouped_family_types['onpromotion'].sum()
data_grouped_family_types['%_p'] = data_grouped_family_types['%_p'].round(decimals=3)

# Convert float16 to float64
data_grouped_family_types['%_p'] = data_grouped_family_types['%_p'].astype('float32')

# Create labels
labels = [f"{category} - {percent:.2f}%" for category, percent in zip(family_category.values(), data_grouped_family_types['%_p'])]

# Create a Pie chart
fig = go.Figure(go.Pie(
    labels=labels,
    values=data_grouped_family_types['%_p'],
    textinfo='none',
    marker=dict(colors=px.colors.sequential.Rainbow),
    sort=True,  # Keep the order as in the DataFrame
))

# Update layout
fig.update_layout(
    title="On Promotion Distribution by Family Category",
    legend=dict(orientation="v", x=-0.8, y=0.2), height=950, width=900
)

# Show the plot
fig.show()

<ipython-input-233-df0592a56933>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



- 카테고리별로 프로모션 여부의 분포를 시각적으로 보여주며, 프로모션 여부에 따라 각 카테고리가 차지하는 백분율을 보여줌
- 가장 많이 판매되고 가장 많은 프로모션을 진행하는 품목은 식료품, 음료, 농산물, 유제품, 청소라는 것을 알 수 있음. 식료품 + 음료가 전체 매출의 50% 이상을 차지함.
- 데이터 집합에는 많은 수의 행이 포함되어 있으므로, 예를 들어 주 또는 월별로 데이터를 그룹화하는 것이 더 간단함.

In [ ]:
def grouped(df, key, freq, col):
    # 주어진 주기에 따라 데이터 그룹화
    # agg 함수를 사용하여 각 그룹에서 지정된 column의 평균 계산
    df_grouped = df.groupby([pd.Grouper(key=key, freq=freq)]).agg(mean = (col, 'mean'))

    # 결과를 새로운 데이터프레임으로 변환하고 인덱스 재설정
    df_grouped = df_grouped.reset_index()
    return df_grouped

# 주어진 데이터프레임을 특정 주기로 그룹화하는 함수 정의
# df: 그룹화할 데이터프레임
# key: 그룹화할 열의 이름
# freq: 그룹화할 주기를 나타내는 문자열 (예: 'D' - 일(day), 'W' - 주(week), 'M' - 월(month))
# col: 그룹화될 열의 데이터를 집계할 열의 이름

In [ ]:
df_grouped_trans_w = grouped(transactions, 'date', 'W', 'transactions')
# 'transactions' 데이터프레임을 주어진 주기('W' - 주(week))로 그룹화하고,
# 'date' 열을 기준으로 각 그룹에서 'transactions' 열의 평균을 계산

- For better forecasting added time column

In [ ]:
def add_time(df, key, freq, col):
    # grouped 함수를 사용하여 주어진 주기(freq)에 따라 데이터를 그룹화하고,
    # 'key' 열을 기준으로 각 그룹에서 'col' 열의 평균을 계산
    df_grouped = grouped(df, key, freq, col)

    # 'time' 열을 새로운 시간 변수로 추가하고,
    # 해당 열에는 각 그룹의 인덱스를 순차적으로 할당
    df_grouped['time'] = np.arange(len(df_grouped.index))

    # 'time'열을 데이터프레임의 두 번째 열로 이동시킴
    column_time = df_grouped.pop('time')
    df_grouped.insert(1, 'time', column_time)
    return df_grouped

# 'time' 열 추가하는 함수 정의
# df: 작업을 수행할 데이터프레임
# key: 그룹화할 열의 이름
# freq: 그룹화할 주기를 나타내는 문자열 (예: 'D' - 일(day), 'W' - 주(week), 'M' - 월(month))
# col: 그룹화될 열의 데이터를 집계할 열의 이름

In [ ]:
# 주와 월 단위로 그룹화된 데이터프레임 생성
df_grouped_train_w = add_time(train, 'date', 'W', 'sales')
df_grouped_train_m = add_time(train, 'date', 'M', 'sales')

In [ ]:
# Function to calculate linear regression line
def linear_reg_line(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    return slope * x + intercept

fig_combined = make_subplots(rows=3, cols=1, subplot_titles=['Transactions (grouped by week)', 'Sales (grouped by week)', 'Sales (grouped by month)'], vertical_spacing=0.09)

# Transactions (grouped by week)
fig_combined.add_trace(go.Scatter(x=df_grouped_trans_w['date'], y=df_grouped_trans_w['mean'], mode='lines+markers', name='Transactions', line=dict(color='grey')),
                       row=1, col=1)

# Sales (grouped by week)
fig_combined.add_trace(go.Scatter(x=df_grouped_train_w['time'], y=df_grouped_train_w['mean'], mode='lines+markers', line=dict(color='lightslategrey'), name='Sales (weekly)'),
                       row=2, col=1)
fig_combined.add_trace(go.Scatter(x=df_grouped_train_w['time'], y=linear_reg_line(df_grouped_train_w['time'], df_grouped_train_w['mean']),
                                 mode='lines', line=dict(color='blue'), name='Regression (weekly)'),
                       row=2, col=1)

# Sales (grouped by month)
fig_combined.add_trace(go.Scatter(x=df_grouped_train_m['time'], y=df_grouped_train_m['mean'], mode='lines+markers', line=dict(color='lightslategrey'), name='Sales (monthly)'),
                       row=3, col=1)
fig_combined.add_trace(go.Scatter(x=df_grouped_train_m['time'], y=linear_reg_line(df_grouped_train_m['time'], df_grouped_train_m['mean']),
                                 mode='lines', line=dict(color='red'), name='Regression (monthly)'),
                       row=3, col=1)

fig_combined.update_layout(height=900, width=900, margin=dict(l=30, r=20, t=30, b=30), showlegend=False)

fig_combined.show()

- 선형 회귀 분석을 사용하여 주와 월 단위로 그룹화된 데이터의 판매량 추세를 시각화

In [ ]:
# Create copies of the data with date as the index
transactions_indexed = transactions.set_index('date').copy()
holidays_indexed = holidays.set_index('date').copy()
oil_indexed = oil.set_index('date').copy()
train_indexed = train.set_index('date').copy()

# 10.Trend

In [ ]:
# 주어진 데이터프레임에서 주기별로 특정 열의 추세를 시각화하는 함수
def plot_trend(data, column_name, frequency, data_name):
    # data: 데이터프레임
    # column_name: 추세를 분석할 열의 이름
    # frequency: 분석할 주기('week', 'month', 'year' 중 하나)
    # data_name: 그래프에 표시할 데이터의 이름
    if frequency == 'week':
        data_resampled = data[column_name].resample('W').sum()
        date_format = '%b %Y'
        display_indices = list(range(0, len(data_resampled), 10))
    elif frequency == 'month':
        data_resampled = data[column_name].resample('M').sum()
        date_format = '%b %Y'
        display_indices = list(range(0, len(data_resampled), 6))
    elif frequency == 'year':
        data_resampled = data[column_name].resample('Y').sum()
        date_format = '%Y'
        display_indices = list(range(0, len(data_resampled), 1))

    fig = make_subplots()

    trace = go.Scatter(x=data_resampled.index, y=data_resampled/1e6, mode='lines', name=data_name, line=dict(color='blue'))
    fig.add_trace(trace)

    fig.update_layout(
        xaxis=dict(title='Date', tickangle=90, tickmode='array',
                   tickvals=data_resampled.index[display_indices] if display_indices else None,
                   ticktext=data_resampled.index[display_indices].strftime(date_format) if display_indices else None,
                   range=[data_resampled.index.min() - pd.Timedelta(days=30), data_resampled.index.max() + pd.Timedelta(days=30)]),
        yaxis=dict(title=data_name, tickformat=".1f", ticksuffix="M"),
        title=f'{data_name} Trend ({frequency}-wise)',
        height=400,
        width=900,
        margin=dict(l=30, r=20, t=30, b=30)
    )

    fig.show()

In [ ]:
# 'transactions_indexed' 데이터프레임에서 'transactions' 열의 주/월/연 단위 추세를 시각화
plot_trend(transactions_indexed, 'transactions', 'week', 'Transactions')
plot_trend(transactions_indexed, 'transactions', 'month', 'Transactions')
plot_trend(transactions_indexed, 'transactions', 'year', 'Transactions')

In [ ]:
plot_trend(train_indexed, 'sales', 'week', 'Sales')
plot_trend(train_indexed, 'sales', 'month', 'Sales')
plot_trend(train_indexed, 'sales', 'year', 'Sales')

# 11.Seasonality

In [ ]:
# - 계절성 데이터를 시각화하는 함수
def seasonal_plotly(X, y, period, freq):
    # X: 데이터프레임
    # y: 종속 변수의 열 이름
    # period: 계절성이 발생하는 주기를 지정하는 문자열("week", "day" 또는 사용자 정의 주기)
    # freq: 주기 내에서 시각화할 시간 단위의 열 이름("week", "day" 또는 사용자 정의 주기)
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

    colors = ['blue', 'green', 'red', 'purple', 'orange']  # Add more colors if needed

    for idx, (group_name, group_data) in enumerate(X.groupby('year')):
        if period == "week":
            hover_text = [f"Day {day}, Week {week}, Year {group_name}, {y} {y_val:.2f} " for week, day, y_val in zip(group_data['week'], group_data[freq], group_data[y])]
        elif period == "day":
            hover_text = [f"Day {day}, Year {group_name}, {y} {y_val:.2f}" for day, y_val in zip(group_data[freq], group_data[y])]
        else:
            hover_text = [f"{freq} {val}, Year {group_name}, {y} {y_val:.2f}" for val, y_val in zip(group_data[freq], group_data[y])]

        fig.add_trace(go.Scatter(x=group_data[freq], y=group_data[y],
                           mode='lines',
                           name=f"Year {group_name}",
                           line=dict(color=colors[idx]),
                           text=hover_text,
                           hoverinfo="text"))

    # Add dropdown menu for year selection
    buttons = [{"label": "All Years",
                "method": "update",
                "args": [{"visible": [True] * len(X['year'].unique())},
                         {"title": f"Seasonal Plot ({period}/{freq}) - All Years",
                          "showlegend": True}]}]

    for idx, year in enumerate(X['year'].unique()):
        visibility = [True if selected_year == year else False for selected_year in X['year'].unique()]
        buttons.append(dict(label=f"{year}",
                            method="update",
                            args=[{"visible": visibility},
                                  {"title": f"Seasonal Plot ({period}/{freq}) - Year {year}",
                                   "showlegend": False}]))

    # Set initial visibility
    initial_visibility = [True] + [False] * (len(X['year'].unique()) - 1)

    # Add a bit of dummy space on the x-axis by extending the x-axis range
    dummy_space = 0.1 if period == "week" else 10
    x_axis_range = [X[freq].min() - dummy_space, X[freq].max() + dummy_space]

    fig.update_layout(updatemenus=[{"buttons": buttons,
                                    "direction": "down",
                                    "showactive": True,
                                    "x": 0.66,
                                    "xanchor": "center",
                                    "y": 1.1,
                                    "yanchor": "top"}],
                      showlegend=True,
                      xaxis_range=x_axis_range)

    fig.update_layout(height=600, margin=dict(l=5, r=10, t=40, b=30),
                      title=f"Seasonal Plot ({period}/{freq}) - All Years")

    fig.update_layout(xaxis_title=f"{freq}", yaxis_title=f"{y}")

    fig.show()

In [ ]:
# 계절성 데이터 시각화

# train 데이터프레임에서 날짜별 평균 판매량을 계산하고 날짜를 인덱스로 설정
avg_sales_train = train.groupby('date').agg({'sales': 'mean'}).reset_index()
avg_sales_train = avg_sales_train.set_index('date').to_period("D")

X = avg_sales_train.copy()

# 날짜의 요일, 주, 일년 중 몇 번째 날인지, 연도 추가
X['day'] = X.index.dayofweek
X['week'] = X.index.week

X['dayofyear'] = X.index.dayofyear
X['year'] = X.index.year

# 주 단위로 주간 판매량의 계절성 시각화. x축: 요일
seasonal_plotly(X, y="sales", period="week", freq="day")

# 연 단위로 연간 판매량의 계절성 시각화. x축: 연중 날짜
seasonal_plotly(X, y="sales", period="year", freq="dayofyear")

In [ ]:
# 거래 데이터를 기반으로 계절성 시각화

# 날짜별 평균 거래량을 계산하고 날짜를 인덱스로 설정
avg_transactions_train = transactions.groupby('date').agg({'transactions': 'mean'}).reset_index()
avg_transactions_train = avg_transactions_train.set_index('date').to_period("D")

X = avg_transactions_train.copy()

# 날짜의 요일, 주, 일년 중 몇 번째 날인지, 연도 추가
X['day'] = X.index.dayofweek
X['week'] = X.index.week

X['dayofyear'] = X.index.dayofyear
X['year'] = X.index.year

# 주 단위로 주간 거래량의 계절성 시각화. x축: 요일
seasonal_plotly(X, y="transactions", period="week", freq="day")
# 연 단위로 연간 거래량의 계절성 시각화. x축: 연중 날짜
seasonal_plotly(X, y="transactions", period="year", freq="dayofyear")

# 12.Did Earthquake affect store sales?

- comparing average sales by year-month

In [ ]:
train['year_month'] = train['date'].dt.to_period('M').astype(str)

# group by year-month and calculate average sales
avg_sales = train.groupby('year_month')['sales'].mean()

In [ ]:
avg_sales

year_month
2013-01    186.952408
2013-02    193.581848
2013-03    206.880585
2013-04    205.639069
2013-05    209.943588
2013-06    218.655899
2013-07    203.783371
2013-08    212.479431
2013-09    220.593582
2013-10    213.164261
2013-11    231.136536
2013-12    295.606384
2014-01    342.341705
2014-02    241.268906
2014-03    368.661224
2014-04    240.577087
2014-05    242.203125
2014-06    249.157471
2014-07    351.578369
2014-08    251.351791
2014-09    374.530792
2014-10    369.213684
2014-11    384.056030
2014-12    455.302185
2015-01    269.666595
2015-02    275.420807
2015-03    282.368622
2015-04    279.743134
2015-05    320.958099
2015-06    404.327728
2015-07    402.042297
2015-08    415.692291
2015-09    434.734039
2015-10    432.248444
2015-11    426.579773
2015-12    509.614319
2016-01    434.050262
2016-02    424.695374
2016-03    418.735382
2016-04    485.653320
2016-05    448.561462
2016-06    415.436188
2016-07    424.725250
2016-08    406.437378
2016-09    419.331238

In [ ]:
avg_sales.index = pd.to_datetime(avg_sales.index)

In [ ]:
# 특정 기간 동안의 평균 판매량을 나타내는 그래프

fig = go.Figure()

fig.add_trace(go.Scatter(x=avg_sales.index, y=avg_sales.values, mode='lines', name='Average Sales'))

earthquake_date = '2016-04-16'

# 빨간 점선 > 2016-04-16: 지진 발생일
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=earthquake_date,
        x1=earthquake_date,
        y0=avg_sales.values.min()-50,
        y1=avg_sales.values.max()+50,
        line=dict(color="red", dash="dash")
    )
)

# Calculate the start and end dates for the 10-day range
start_date = '2012-11-1'
end_date = '2017-8-30'

fig.update_layout(
    xaxis=dict(title='Year-Month', range=[start_date, end_date]),
    yaxis=dict(title='Sales'),
    title='Average Sales Over Time',
    showlegend=False,
    height=400,
    margin=dict(l=5, r=10, t=40, b=30)
)

fig.show()

# 13.Machine Learning Forecasting

In [ ]:
train['week'] = train['date'].dt.isocalendar().week
train['day'] = train['date'].dt.day

test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['week'] = test['date'].dt.isocalendar().week

In [ ]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,family_category,day_of_week,month,year,year_month,week,day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,0,1,1,2013,2013-01,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0.0,1,1,1,2013,2013-01,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0.0,2,1,1,2013,2013-01,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,3,1,1,2013,2013-01,1,1
4,4,2013-01-01,1,BOOKS,0.0,0.0,4,1,1,2013,2013-01,1,1


In [ ]:
test.head()

,id,date,store_nbr,family,onpromotion,year,month,day,week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,2017,8,16,33
1,3000889,2017-08-16,1,BABY CARE,0,2017,8,16,33
2,3000890,2017-08-16,1,BEAUTY,2,2017,8,16,33
3,3000891,2017-08-16,1,BEVERAGES,20,2017,8,16,33
4,3000892,2017-08-16,1,BOOKS,0,2017,8,16,33


In [ ]:
train = train.drop(['day_of_week','family_category', 'year_month'], axis=1)
train.head()

,id,date,store_nbr,family,sales,onpromotion,month,year,week,day
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,1,2013,1,1
1,1,2013-01-01,1,BABY CARE,0.0,0.0,1,2013,1,1
2,2,2013-01-01,1,BEAUTY,0.0,0.0,1,2013,1,1
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,1,2013,1,1
4,4,2013-01-01,1,BOOKS,0.0,0.0,1,2013,1,1


In [ ]:
test.head()

,id,date,store_nbr,family,onpromotion,year,month,day,week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,2017,8,16,33
1,3000889,2017-08-16,1,BABY CARE,0,2017,8,16,33
2,3000890,2017-08-16,1,BEAUTY,2,2017,8,16,33
3,3000891,2017-08-16,1,BEVERAGES,20,2017,8,16,33
4,3000892,2017-08-16,1,BOOKS,0,2017,8,16,33


In [ ]:
train['week'] = train['week'].astype('int')
test['week'] = test['week'].astype('int')

In [ ]:
# !pip install feature_engine

In [ ]:
# MACHINE LEARNING
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from feature_engine.encoding import OrdinalEncoder
from feature_engine.creation import CyclicalFeatures
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import FunctionTransformer

from xgboost import XGBRegressor
from tqdm.auto import tqdm

## 1) Data 분리

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(['id','date', 'sales'], axis=1), train['sales'], test_size = 0.3)
# X: id, date, sales 열 제외한 나머지
# y: train['sales']

In [ ]:
cat_variables=['family'] # 범주형 변수
time_variables = ['day', 'month', 'year','week'] # 시간 변수

# 범주형 변수에 순여 부여
ord_encoder=OrdinalEncoder(encoding_method='arbitrary', variables=cat_variables)

# 시간 변수를 주기적 특성으로 변환
cf_encoder = CyclicalFeatures(variables=time_variables)

encoders = make_pipeline(
    (ord_encoder), # Ordinal Encoder
    (cf_encoder), # Cyclical Feature
)

## 2) Transformer

In [ ]:
# XGBoost 모델 초기화
xgb_params = {'n_estimators': 100, 'max_depth': 6, 'eta': 0.1, 'gamma': 2, 'n_jobs': -1, 'random_state': 42, 'eval_metric': 'rmse'}
xgb = XGBRegressor(**xgb_params)

# n_estimators: 생성할 트리의 개수 지정
# max_depth: 각 트리의 최대 깊이 지정
# eta: 학습률. 각 단계에서 얼마나 많은 가중치를 적용할지 결정
# gamma: 분할을 수행하는데 필요한 최소 손실 감소 지정. 이 값보다 적은 감소가 발생하면 추가 분할을 수행하지 않음.
# n_jobs: 사용할 코어 수. -1로 설정하면 가능한 모든 코어를 사용
# random_state: 모델의 랜덤 시드를 지정하여 재현성 보장
# eval_metric: 모델의 평가 메트릭으로, RMSE(Root Mean Squared Error)를 사용

In [ ]:
# 모델 학습을 간편하게 수행하기 위해 파이프라인 구축
pipeline = make_pipeline(
    (encoders), # Encoders
    (xgb) # XGBoost
)
print("Pipeline Created!")

Pipeline Created!


In [ ]:
transformer = TransformedTargetRegressor(regressor=pipeline, func=np.log1p, inverse_func=lambda x: np.exp(x) - 1,check_inverse=False)

- regressor: 파이프라인(pipeline)이 지정됩니다. 이 파이프라인은 데이터 전처리 및 모델 학습을 담당합니다.
- func: 타겟 변수를 변환하는 함수가 지정됩니다. 이 경우 로그 변환(log1p)이 사용됩니다. 로그 변환은 종속 변수의 분포를 정규분포에 가깝게 만들어 선형 모델이나 모형의 정확도를 향상시킬 수 있습니다.
- inverse_func: 타겟 변수를 역변환하는 함수가 지정됩니다. 이 경우 로그 역변환(exp(x) - 1)이 사용됩니다.
- check_inverse: 역변환의 유효성을 검사할지 여부를 결정합니다. False로 설정하면 역변환 함수를 적용할 때 경고가 발생하지 않습니다.

## 3) 교차검증

In [ ]:
tscv = TimeSeriesSplit(n_splits=5, gap=7*54*33)

rmse_values = []
rmsle_values = []

cv = 1

# 교차 검증
print("Cross Validating.....")

for train_index, test_index in tscv.split(X_train, y_train):

    # train_index와 test_index를 사용하여 학습 및 테스트 데이터 생성
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    # 모델 학습
    transformer.fit(X_train_cv, y_train_cv)

    # 모델 예측
    y_pred = transformer.predict(X_test_cv)

    # RMSE, RMSLE 계산
    rmse = mean_squared_error(y_test_cv, y_pred, squared=False)
    rmsle = mean_squared_log_error(y_test_cv, y_pred.clip(0.0), squared=False)

    print(f"cv #{cv}: RMSE = {rmse:.3f} | RMSLE = {rmsle:.3f}")

    rmse_values.append(rmse)
    rmsle_values.append(rmsle)
    cv += 1

print(f"Average RMSE: {np.mean(rmse_values):.3f} | Average RMSLE: {np.mean(rmsle_values):.3f}")

Cross Validating.....
cv #1: RMSE = 599.616 | RMSLE = 0.926
cv #2: RMSE = 620.546 | RMSLE = 0.938
cv #3: RMSE = 613.745 | RMSLE = 0.936
cv #4: RMSE = 585.773 | RMSLE = 0.930
cv #5: RMSE = 590.151 | RMSLE = 0.947
Average RMSE: 601.966 | Average RMSLE: 0.936


In [ ]:
# 모델 평가

# 예측값 계산
predictions = transformer.predict(X_val)

val_rmse = mean_squared_error(y_val, predictions, squared=False)
val_rmsle = mean_squared_log_error(y_val, predictions.clip(0.0), squared=False)

print(f"Test RMSE Score: {val_rmse:.3f} | Test RMSLE Score: {val_rmsle:.3f}")

Test RMSE Score: 605.923 | Test RMSLE Score: 0.945


In [ ]:
# validation data에 대한 예측값과 실제값을 포함하는 데이터프레임을 만들어 예측 오차 게산

# 검증 데이터셋 X_val의 'store_nbr' 및 'family' 열과 실제값 y_val을 합친 데이터프레임 생성
val_df = pd.concat([X_val[['store_nbr', 'family']], y_val], axis=1)

# 앞서 계산한 예측값을 val_df에 추가
val_df['predictions'] = predictions

# 임계일 설정
cut_off_date = "2017-06-15"

# 임계일 이후의 모든 날짜 가져옴
val_df['date'] = train.loc[train['date'].ge(cut_off_date), 'date']

# 예측 오차 계산 후 error 열에 저장
val_df['error'] = val_df['sales'] - val_df['predictions']

In [ ]:
val_df.head()

,store_nbr,family,sales,predictions,date,error
1351479,29,PRODUCE,0.0,2.749571,NaT,-2.749571
1980982,41,PERSONAL CARE,218.0,259.039612,NaT,-41.039612
321565,31,GROCERY II,11.0,4.027178,NaT,6.972822
2076041,1,FROZEN FOODS,93.0,117.271240,NaT,-24.271240
2380919,14,BEAUTY,0.0,0.925718,NaT,-0.925718


In [ ]:
fig = px.line(
    # 날짜별로 실제 판매량('sales')과 예측된 판매량('predictions')을 합산하고, 이를 다시 데이터프레임으로 재구성
    val_df.groupby('date')[['sales', 'predictions']].sum().reset_index(),
    x='date', y=['sales', 'predictions'], title='Actual Total Sales vs Predicted Total Sales')

fig.update_layout(height=500,margin=dict(l=20, r=20, t=50, b=10), legend_title='')

fig.show()

- 실제 판매량과 예측된 판매량의 추이 비교 그래프

---

In [ ]:
# 날짜와 가족별로 실제 판매량과 예측된 판매량을 합산한 데이터프레임을 생성
family_agg_val_df = val_df.groupby(['date', 'family'], as_index=False)[['sales', 'predictions']].sum()

fig = make_subplots(rows=17, cols=2, shared_xaxes=False, vertical_spacing=0.015, subplot_titles=family_agg_val_df['family'].unique())

for i,prd in enumerate(family_agg_val_df['family'].unique(), start=1):

    subset_df = family_agg_val_df.query(f"family=='{prd}'")
    dynamic_row = (i + 1) // 2
    if i%2==1:
        # Add sales trace
        fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['sales'], mode='lines', name='Sales', line=dict(color='blue'), legendgroup='Sales', showlegend=True if i==1 else False), row=dynamic_row, col=1)
        # Add predictions trace
        fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['predictions'], mode='lines', name='Predictions', line=dict(color='red'), legendgroup='Predictions', showlegend=True if i==1 else False), row=dynamic_row, col=1)
    else:
        # Add sales trace
        fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['sales'], mode='lines', name='Sales', line=dict(color='blue'), legendgroup='Sales', showlegend=False), row=dynamic_row, col=2)
        # Add predictions trace
        fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['predictions'], mode='lines', name='Predictions', line=dict(color='red'), legendgroup='Predictions', showlegend=False), row=dynamic_row, col=2)

fig.update_layout(height=5000, width=910, title_text="Family-wise Sales and Predictions Over Time", margin=dict(l=10, r=10, t=100, b=10),
                  legend=dict(x=0.9, y=1.017, bgcolor='rgba(255, 255, 255, 0.5)'))

fig.show()

- 각 가족별로 판매 및 예측 추이 비교 그래프

---

- test data predictions and graphs

In [ ]:
test.head()

,id,date,store_nbr,family,onpromotion,year,month,day,week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,2017,8,16,33
1,3000889,2017-08-16,1,BABY CARE,0,2017,8,16,33
2,3000890,2017-08-16,1,BEAUTY,2,2017,8,16,33
3,3000891,2017-08-16,1,BEVERAGES,20,2017,8,16,33
4,3000892,2017-08-16,1,BOOKS,0,2017,8,16,33


In [ ]:
new_df = pd.DataFrame({"date": test["date"]})
X_test = test.drop(['id','date'], axis=1)

In [ ]:
X_test.head()

,store_nbr,family,onpromotion,year,month,day,week
0,1,AUTOMOTIVE,0,2017,8,16,33
1,1,BABY CARE,0,2017,8,16,33
2,1,BEAUTY,2,2017,8,16,33
3,1,BEVERAGES,20,2017,8,16,33
4,1,BOOKS,0,2017,8,16,33


In [ ]:
X_test = X_test.reindex(columns=['store_nbr', 'family', 'onpromotion', 'month', 'year', 'week', 'day'])

In [ ]:
X_test.head()

,store_nbr,family,onpromotion,month,year,week,day
0,1,AUTOMOTIVE,0,8,2017,33,16
1,1,BABY CARE,0,8,2017,33,16
2,1,BEAUTY,2,8,2017,33,16
3,1,BEVERAGES,20,8,2017,33,16
4,1,BOOKS,0,8,2017,33,16


In [ ]:
predictions = transformer.predict(X_test)

In [ ]:
test_df = pd.DataFrame({
    "store_nbr": X_test["store_nbr"],
    "family": X_test["family"]
})
test_df['predictions'] = predictions
cut_off_date = "2017-06-15"
test_df['date'] = new_df

In [ ]:
test_df.head()

,store_nbr,family,predictions,date
0,1,AUTOMOTIVE,6.952089,2017-08-16
1,1,BABY CARE,0.262661,2017-08-16
2,1,BEAUTY,5.639384,2017-08-16
3,1,BEVERAGES,2000.703125,2017-08-16
4,1,BOOKS,-0.005172,2017-08-16


In [ ]:
fig = px.line(test_df.groupby('date')['predictions'].sum().reset_index(),
              x='date',
              y=['predictions'],
              title='Predicted Total Sales')

fig.update_layout(height=500,margin=dict(l=20, r=20, t=50, b=10), legend_title='')

fig.show()

- 날짜별 예측된 매출 그래프

---

In [ ]:
family_agg_val_df = test_df.groupby(['date', 'family'], as_index=False)['predictions'].sum()

fig = make_subplots(rows=17, cols=2, shared_xaxes=False, vertical_spacing=0.015, subplot_titles=family_agg_val_df['family'].unique())

for i,prd in enumerate(family_agg_val_df['family'].unique(), start=1):

    subset_df = family_agg_val_df.query(f"family=='{prd}'")
    dynamic_row = (i + 1) // 2
    if i%2==1:
        # Add predictions trace
        fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['predictions'], mode='lines', name='Predictions', line=dict(color='red')), row=dynamic_row, col=1)
    else:
        # Add predictions trace
        fig.add_trace(go.Scatter(x=subset_df['date'], y=subset_df['predictions'], mode='lines', name='Predictions', line=dict(color='red')), row=dynamic_row, col=2)

fig.update_layout(height=5000, width=910, title_text="Family-wise Predictions Over Time", showlegend=False, margin=dict(l=10, r=10, t=100, b=10))

fig.show()

- 각 가족별로 예측된 매출을 시간에 따라 보여주는 그래프